# Impact of queue length considered on Saturation Flow Rate
Created by: Apoorb  
Date: Dec 9, 2019

## Load Libraries

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
os.chdir(r'C:\Users\abibeka\OneDrive - Kittelson & Associates, Inc\Documents\HCM-CAV-Pooled-Fund\VissimBaseModel')
os.getcwd()

'C:\\Users\\abibeka\\OneDrive - Kittelson & Associates, Inc\\Documents\\HCM-CAV-Pooled-Fund\\VissimBaseModel'

In [2]:
RawData = pd.read_csv("RawData.csv")
RawData = RawData[['CycNum','Lane','tQueue','t_Entry','G_st','G_end','VehNum','PhaseNum']]


In [3]:
def AvgHeadway_by_Queue(Dat, MaxVnum = 8, MinVnum = 5):
    mask = (Dat.VehNum >= MinVnum) & (Dat.VehNum <= MaxVnum) 
    Dat = Dat[mask]
    Dat = Dat.groupby(['CycNum','Lane']).agg({'t_Entry':['min','max'],'VehNum':['min','max']})
    Dat.columns = ['_'.join(col).strip() for col in Dat.columns.values]
    Dat.loc[:,'AvgHeadway'] = (Dat.t_Entry_max - Dat.t_Entry_min)/(Dat.VehNum_max - Dat.VehNum_min)
    idx = pd.IndexSlice
#Average headway lane 1
    AvgSatFlowLn1 = int(np.round(3600/Dat.loc[idx[:,1],'AvgHeadway'].mean(),0))
#Average headway lane 2
    AvgSatFlowLn2 = int(np.round(3600/Dat.loc[idx[:,2],'AvgHeadway'].mean(),0))
# Max veh in queue
    
    return ([AvgSatFlowLn1,AvgSatFlowLn2,Dat])

In [4]:
MaxQLen = range(8, 25)

In [5]:
DictSatFlow = {}
for i in MaxQLen:
    DictSatFlow[i] = AvgHeadway_by_Queue(RawData,i, 5)[:2]

In [6]:
DatQLen =pd.DataFrame(DictSatFlow).T.reset_index()
DatQLen.rename(columns={'index':'MaxQLenVeh',0:'SatFlowLane1',1:'SatFlowLane2'}, inplace=True)
DatQLen.loc[:,'QStartsAt'] = "5th_Veh"

In [7]:
DatQLen.to_csv('SatFlow_by_MaxQueueConsidered.csv',index=False)